# Movie Review Explanations


We will use an SKLearn classifier built on [movie sentiment data](http://www.cs.cornell.edu/people/pabo/movie%2Dreview%2Ddata/) which predicts positive or negative sentiment for review text. 

The Kfserving resource provdes:
   * A pretrained sklearn model stored on a Google bucket
   * A Text [Seldon Alibi](https://github.com/SeldonIO/alibi) Explainer. See the [Alibi Docs](https://docs.seldon.io/projects/alibi/en/stable/) for further details.

In [1]:
!pygmentize moviesentiment.yaml

apiVersion: "serving.kubeflow.org/v1alpha2"
kind: "InferenceService"
metadata:
  name: "moviesentiment"
spec:
  default:
    predictor:
      minReplicas: 1
      sklearn:
        storageUri: "gs://seldon-models/sklearn/moviesentiment"
        resources:
          requests:
            cpu: 0.1
            memory: 1Gi                        
          limits:
            cpu: 1
            memory: 1Gi                        
    explainer:
      minReplicas: 1
      alibi:
        type: AnchorText
        resources:
          requests:
            cpu: 0.1
            memory: 6Gi            
          limits:
            memory: 6Gi


In [2]:
!kubectl apply -f moviesentiment.yaml

inferenceservice.serving.kubeflow.org/moviesentiment configured


In [3]:
CLUSTER_IPS=!(kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}')
CLUSTER_IP=CLUSTER_IPS[0]
print(CLUSTER_IP)

35.204.158.239


In [4]:
SERVICE_HOSTNAMES=!(kubectl get inferenceservice moviesentiment -o jsonpath='{.status.url}' | cut -d "/" -f 3)
SERVICE_HOSTNAME=SERVICE_HOSTNAMES[0]
print(SERVICE_HOSTNAME)

moviesentiment.default.example.com


In [5]:
import sys
sys.path.append('../')
from alibi_helper import *

In [6]:
from alibi.datasets import fetch_movie_sentiment
movies = fetch_movie_sentiment()

In [107]:
idxNeg = 37
idxPos = 5215
for idx in [idxNeg,idxPos]:
    print(movies.data[idx])
    show_prediction(predict(movies.data[idx:idx+1],'moviesentiment',movies,SERVICE_HOSTNAME,CLUSTER_IP))

curling may be a unique sport but men with brooms is distinctly ordinary .


## Prediction: negative

if you sometimes like to go to the movies to have fun , wasabi is a good place to start .


## Prediction: positive

# Get Explanation for Negative Prediction

In [108]:
exp = explain(movies.data[idxNeg:idxNeg+1],"moviesentiment",SERVICE_HOSTNAME,CLUSTER_IP)

In [109]:
show_anchors(exp['names'])

# Explanation:

## ['distinctly', 'ordinary']

Show precision. How likely predictions using the Anchor features would produce the same result.

In [110]:
show_bar([exp['precision']],[''],"Precision")
show_bar([exp['coverage']],[''],"Coverage")

In [111]:
show_feature_coverage(exp)

In [112]:
show_examples(exp,0,movies)

## Examples covered by Anchors: ['distinctly']

,0
0,curling may be UNK UNK UNK UNK UNK UNK brooms ...
1,curling UNK UNK a unique sport but UNK with br...
2,curling UNK UNK a unique sport UNK UNK with br...
3,curling UNK UNK UNK UNK sport but men with UNK...
4,curling may UNK a UNK UNK but UNK with UNK is ...
5,curling UNK be a unique sport but men with UNK...
6,curling UNK UNK a unique UNK UNK men UNK UNK U...
7,curling UNK UNK a UNK sport but men with UNK i...
8,UNK UNK UNK a unique sport but men UNK UNK UNK...
9,UNK may be UNK unique sport UNK UNK UNK brooms...


In [113]:
show_examples(exp,0,movies,False)

## Examples not covered by Anchors: ['distinctly']

,0
0,UNK UNK UNK a unique UNK UNK UNK with UNK is d...
1,UNK may UNK UNK unique UNK but men with UNK is...
2,UNK may be a unique sport but UNK with UNK is ...
3,curling UNK UNK UNK unique UNK UNK UNK with UN...
4,UNK may UNK UNK unique sport but men with UNK ...
5,curling UNK UNK a unique UNK UNK UNK with UNK ...
6,UNK may be UNK unique sport UNK men with UNK U...
7,UNK may UNK UNK unique UNK UNK men UNK UNK is ...
8,UNK may UNK UNK unique UNK UNK UNK with UNK is...
9,UNK may be UNK unique sport but men with UNK i...


# Get Explanation for High Income Example

In [114]:
exp = explain(movies.data[idxPos:idxPos+1],"moviesentiment",SERVICE_HOSTNAME,CLUSTER_IP)

In [115]:
show_anchors(exp['names'])

# Explanation:

## ['fun', 'good']

Show precision. How likely predictions using the Anchor features would produce the same result.

In [116]:
show_bar([exp['precision']],[''],"Precision")
show_bar([exp['coverage']],[''],"Coverage")

In [117]:
show_feature_coverage(exp)

In [120]:
show_examples(exp,0,movies)

## Examples covered by Anchors: ['fun']

,0
0,if you UNK UNK UNK UNK UNK UNK UNK UNK UNK fun...
1,if UNK UNK UNK to go UNK UNK movies UNK have f...
2,UNK you sometimes like UNK go UNK the UNK to U...
3,UNK you sometimes UNK UNK UNK to UNK movies to...
4,if you UNK UNK UNK UNK UNK the UNK UNK UNK fun...
5,UNK UNK sometimes UNK UNK go UNK the movies UN...
6,if you UNK like to go UNK the movies UNK have ...
7,UNK UNK sometimes UNK to go to the movies UNK ...
8,if you UNK like to go UNK the UNK to have fun ...
9,UNK you sometimes like UNK go to UNK movies UN...


In [119]:
show_examples(exp,0,movies,False)

## Examples not covered by Anchors: ['fun']

,0
0,UNK you UNK like to go to UNK UNK UNK have fun...
1,if UNK sometimes UNK UNK go to UNK UNK UNK hav...
2,if UNK sometimes like to UNK to the UNK to hav...
3,UNK UNK sometimes like to go to the UNK to hav...
4,UNK UNK UNK UNK to UNK UNK the movies to have ...
5,UNK you sometimes like to go UNK UNK UNK to ha...
6,UNK UNK UNK UNK to UNK to the UNK UNK have fun...
7,UNK you UNK like to UNK to the UNK UNK have fu...
8,UNK UNK UNK UNK UNK go to the UNK to have fun ...
9,UNK UNK sometimes like UNK UNK to the UNK UNK ...
